Music Rec System

In [111]:
import sys
print(sys.executable)


/Users/anvikudaraya/miniconda3/envs/songrec/bin/python


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("spotify_millsongdata.csv")

In [5]:
df.isnull().sum() 
#check for any missing vals

artist    0
song      0
link      0
text      0
dtype: int64

In [66]:
df = df.sample(n=min(10000, len(df)), random_state=42).drop('link', axis=1, errors='ignore').reset_index(drop=True)

In [67]:
df.head(3)

,artist,song,text
0,Gino Vannelli,Stay With Me,hey woman walking on a wire \r i know your ba...
1,Michael W. Smith,The Happiest Christmas,\r oh the happiest christmas \r is a homeco...
2,Bruno Mars,Long Distance,there's only so many songs that i can sing \r...


In [68]:
df['text'][0]

"hey woman walking on a wire  \r i know your back is to the wall  \r what lies in water lies in fire  \r it's all in how you take the fall  \r why don't you stay with me  \r oh, lay with me  \r don't turn the lights on me  \r tonight on me  \r (stay with me)  \r the streets are cold in new york city  \r why don't we break the ice  \r now just like a fool i'll be feeling guilty  \r so just this once i won't think twice  \r why don't you stay with me  \r oh, lay with me  \r the only crime i see  \r is killing time with me  \r (stay with me)  \r and oh, what a feeling  \r when i'm with you  \r you play with fire  \r what am i supposed to do  \r hey woman life's too short to straddle  \r you're either right or wrong  \r so come on gamble  \r just don't keep me hanging on  \r why don't you stay with me  \r oh lay with me  \r the only crime i see  \r is killing time with me  \r \r "

In [96]:
#text cleaning and preprocessing
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex=True)

In [98]:
df.head()

,artist,song,text
0,Gino Vannelli,Stay With Me,hey woman walking on a wire \r i know your ba...
1,Michael W. Smith,The Happiest Christmas,\r oh the happiest christmas \r is a homeco...
2,Bruno Mars,Long Distance,there's only so many songs that i can sing \r...
3,Boney M.,Baby Do You Wanna Bump,do you want to bump? \r baby do you want to b...
4,Kylie Minogue,Extraordinary Day,"when i'm all alone, dreaming of where i want t..."


In [70]:
df

,artist,song,text
0,Gino Vannelli,Stay With Me,hey woman walking on a wire \r i know your ba...
1,Michael W. Smith,The Happiest Christmas,\r oh the happiest christmas \r is a homeco...
2,Bruno Mars,Long Distance,there's only so many songs that i can sing \r...
3,Boney M.,Baby Do You Wanna Bump,do you want to bump? \r baby do you want to b...
4,Kylie Minogue,Extraordinary Day,"when i'm all alone, dreaming of where i want t..."
...,...,...,...
4995,Moody Blues,Boulevard De La Madeleine,"she said she'd come, she didn't \r i'm the on..."
4996,Fleetwood Mac,Empire State,written by lindsey buckingham. \r \r new yo...
4997,Kenny Chesney,Small Y'all,"honey, you think he's got an attitude \r so y..."
4998,Devo,Social Fools,if you obey society's rules \r you will be so...


In [71]:
import nltk 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [72]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anvikudaraya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [73]:
stemmer = PorterStemmer()

In [74]:
import nltk

# robust: try both, since newer NLTK may also need 'punkt_tab'
for pkg in ("punkt", "punkt_tab"):
    try:
        nltk.data.find(f"tokenizers/{pkg}")
    except LookupError:
        nltk.download(pkg)

In [102]:
#turns text into toekn
def token(txt):
    token = nltk.word_tokenize(txt)
    stems = [stemmer.stem(w) for w in token]
    return " ".join(stems)

In [76]:
print(token("you are beautiful, beauty"))

you are beauti , beauti


In [103]:
df['text'].apply(lambda x: token(x))

0       hey woman walk on a wire i know your back is t...
1       oh the happiest christma is a homecom christma...
2       there 's onli so mani song that i can sing to ...
3       do you want to bump ? babi do you want to bump...
4       when i 'm all alon , dream of where i want to ...
                              ...                        
4995    she said she 'd come , she did n't i 'm the on...
4996    written by lindsey buckingham . new york , ope...
4997    honey , you think he 's got an attitud so you ...
4998    if you obey societi 's rule you will be societ...
4999    i got an old slouch hat got my roll on my shou...
Name: text, Length: 5000, dtype: object

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [105]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [106]:
matrix = tfid.fit_transform(df['text'])

In [107]:
similiarity = cosine_similarity(matrix) 

In [82]:
similiarity[0]

array([1.        , 0.02229493, 0.17602988, ..., 0.11329397, 0.01439117,
       0.03895389], shape=(5000,))

In [83]:
df[df['song'] == "loving him was red"]

,artist,song,text


In [108]:
#Recommendation function
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similiarity[idx])),reverse=True,key=lambda x:x[1])
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
    return songs


In [109]:
recommendation('Long Distance')

['Letting You Go',
 'My Piano',
 'Struck In My Heart',
 'Hard',
 'I Hate The Tv',
 'Distance',
 'Sing Together',
 'So Hard',
 'Gypsy',
 'So Close',
 'How Long',
 'Far Away From Home',
 'Killing Me Softly With Her Song',
 'A Change Is Gonna Come',
 'Hard Times For An Honest Man',
 'Moving Oleta',
 'Crying Time',
 'The Distance',
 "It's Over Now",
 'Hard Feelings']

In [110]:
import pickle
pickle.dump(similiarity, open('similiarity.pkl', 'wb'))
pickle.dump(df, open('df.pkl', 'wb'))